I called SNPs across all species. Now, for each species, find the major allele to create a pseudo reference where the original loblolly reference allele is replaced by the species-specific major allele.

Afterward, remap and call SNPs for each species.

In [1]:
from pythonimports import *
DIR = '/lu213/brandon.lind/data/BURT/data'
vcfdir = op.join(DIR, '04_vcfs')
lview,dview = get_client()

56 56


# first set all species to their own vcf file

In [2]:
# get a list of samples
samp2pool = pklload(op.join(DIR, 'samp2pool.pkl'))
len(samp2pool), keys(samp2pool)[0]

(1135, 'T-AR-2-7')

In [9]:
# assign samples to species
species = defaultdict(list)
for samp in samp2pool:
    spp = samp.split("-")[0]
    species[spp].append(samp)
for spp,samplist in species.items():
    print(spp, len(samplist))

T 432
G 171
P 306
E 226


In [12]:
# write samples to species-specific files to use in `bcftools view` call
files = {}
for spp,samplist in species.items():
    file = op.join(vcfdir, f"{spp}_samplist.txt")
    with open(file, 'w') as o:
        o.write('\n'.join(samplist))
    print(spp, file)
    files[spp] = file
len(files)

T /lu213/brandon.lind/data/BURT/data/04_vcfs/T_samplist.txt
G /lu213/brandon.lind/data/BURT/data/04_vcfs/G_samplist.txt
P /lu213/brandon.lind/data/BURT/data/04_vcfs/P_samplist.txt
E /lu213/brandon.lind/data/BURT/data/04_vcfs/E_samplist.txt


4

In [35]:
# get bcftools/vcftools command to filter allsnps.vcf for species sample list, biallelic SNPs, 
    # with min ALT freq of 0.7 and max missing data of 50%
    # the reason I'm using AF > 0.7 is because these are the sites I will want to replace in the reference
# write commands to an executable file
vcf = op.join(vcfdir, 'TotalRawSNPs.vcf.gz')
vcftools = '/data/programs/vcftools_0.1.13/bin/vcftools'
bcftools = '/data/programs/bcftools-1.9/bcftools'
filtfiles = []
for spp,file in files.items():
    sppvcf = vcf.replace('.vcf.gz', f'_alt-filtered_{spp}.vcf.gz')
    # use bcftools to subset samples, then vcftools to do the filtering
    cmd = f"{bcftools} view \
--samples-file {file} {vcf} \
-Oz | {vcftools} --gzvcf - \
--non-ref-af 0.7 \
--max-missing 0.5 \
--minQ 20 \
--recode \
--recode-INFO-all \
--remove-indels \
--min-alleles 2 \
--max-alleles 2 \
--stdout | bgzip -c > {sppvcf}"
    cmdfile = op.join(vcfdir, f'{spp}_filt_cmd.sh')
    with open(cmdfile, 'w') as o:
        o.write(cmd)
    !chmod +x $cmdfile
    print(f'.{cmdfile}')
    filtfiles.append(sppvcf)
len(filtfiles)

./lu213/brandon.lind/data/BURT/data/04_vcfs/T_filt_cmd.sh
./lu213/brandon.lind/data/BURT/data/04_vcfs/G_filt_cmd.sh
./lu213/brandon.lind/data/BURT/data/04_vcfs/P_filt_cmd.sh
./lu213/brandon.lind/data/BURT/data/04_vcfs/E_filt_cmd.sh


4

In [32]:
# I executed these files in separate terminal windows

# convert filtered vcfs to .txt

In [43]:
# index files
gatk = '/data/programs/gatk-4.1.0.0/gatk'
for f in filtfiles:
    cmd = f'{gatk} IndexFeatureFile -F {f}'
    print(cmd, '\n')

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T.vcf.gz 

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G.vcf.gz 

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_P.vcf.gz 

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_E.vcf.gz 



In [48]:
# add in contig lengths so VariantsToTable works below

# I used a new conda env ("picard") to install picard (Version:2.23.8) via bioconda
    # I only used this env for this command.
ref = '/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.fa'
updated = []
for f in filtfiles:
    out = f.replace(".vcf.gz", "_added-lengths.vcf.gz")
    cmd = f"picard UpdateVcfSequenceDictionary I={f} O={out} SEQUENCE_DICTIONARY={ref}"
    print(cmd, '\n')
    updated.append(out)
len(updated)

picard UpdateVcfSequenceDictionary I=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T.vcf.gz O=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T_added-lengths.vcf.gz SEQUENCE_DICTIONARY=/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.fa 

picard UpdateVcfSequenceDictionary I=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G.vcf.gz O=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G_added-lengths.vcf.gz SEQUENCE_DICTIONARY=/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.fa 

picard UpdateVcfSequenceDictionary I=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_P.vcf.gz O=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_P_added-lengths.vcf.gz SEQUENCE_DICTIONARY=/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.fa 

picard UpdateVcfSequenceDictionary I=/lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_E.vcf.gz O=/lu213/brandon.lind

4

In [52]:
# index the new files
for f in updated:
    cmd = f'{gatk} IndexFeatureFile -F {f}'
    print(cmd, '\n')

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T_added-lengths.vcf.gz 

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G_added-lengths.vcf.gz 

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_P_added-lengths.vcf.gz 

/data/programs/gatk-4.1.0.0/gatk IndexFeatureFile -F /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_E_added-lengths.vcf.gz 



In [53]:
# convert to tables
outfiles = []
for f in updated:
    outfile = f.replace(".vcf.gz", ".txt")
    cmd = f'{gatk} VariantsToTable --variant {f} -F CHROM -F POS -F REF -F ALT -O {outfile}'
    print(cmd, '\n')
    outfiles.append(outfile)

/data/programs/gatk-4.1.0.0/gatk VariantsToTable --variant /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T_added-lengths.vcf.gz -F CHROM -F POS -F REF -F ALT -O /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T_added-lengths.txt 

/data/programs/gatk-4.1.0.0/gatk VariantsToTable --variant /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G_added-lengths.vcf.gz -F CHROM -F POS -F REF -F ALT -O /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G_added-lengths.txt 

/data/programs/gatk-4.1.0.0/gatk VariantsToTable --variant /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_P_added-lengths.vcf.gz -F CHROM -F POS -F REF -F ALT -O /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_P_added-lengths.txt 

/data/programs/gatk-4.1.0.0/gatk VariantsToTable --variant /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_E_added-lengths.vcf.gz -F CHROM -F POS -F REF -

In [ ]:
# i manually checked that the new txt files have the same number of sites as the vcf.gz files
    # zcat vcf | grep -v "#" | wc -l
    # wc -l txt

# create psuedo genomes

For all of the sites that I've filtered on a per-species basis above, replace the REF allele with the ALT allele in the reference and save to a new ref.fa file (ie a psuedogenome.fa)

modified from https://github.com/poojasingh09/pseudogenome-maker/blob/master/pseudogenome_maker.sh

credit also to Mengmeng Lu

#### first create a tabbed ref

In [55]:
# convert .fa to .tab
tabbed_ref = ref.replace('.fa', '.tab')
print(f'seqkit fx2tab {ref} > {tabbed_ref}')

seqkit fx2tab /data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.fa > /data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.tab


In [65]:
# double check that ref and alt differ for all loci in the txt files
    # they should since the ALT is what I want to replace in the REF
for f in nb(outfiles):
    df = pd.read_table(f)
    assert sum(df['REF']==df['ALT']) == 0

100%|██████████| 4/4 [00:00<00:00, 23.32it/s]


In [66]:
# replace the REF allele in reference with the ALT allele (AF > 0.7) for each species
for f in outfiles:
    spp = f.split("_")[-2]
    pseudo = op.join(vcfdir, f'psuedogenome_{spp}.fa')
    cmd = '''awk 'FILENAME=="%(tabbed_ref)s" {fa[$1]=$2; next} {fa[$1]=substr(fa[$1], 1, $2-1) $4 substr(fa[$1], $2+1, length(fa[$1])-$2)} END {for (id in fa){print ">" id "\\n" fa[id]}}' %(tabbed_ref)s %(f)s > %(pseudo)s''' % locals()
    print(cmd, '\n')
    

awk 'FILENAME=="/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.tab" {fa[$1]=$2; next} {fa[$1]=substr(fa[$1], 1, $2-1) $4 substr(fa[$1], $2+1, length(fa[$1])-$2)} END {for (id in fa){print ">" id "\n" fa[id]}}' /data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.tab /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_T_added-lengths.txt > /lu213/brandon.lind/data/BURT/data/04_vcfs/psuedogenome_T.fa 

awk 'FILENAME=="/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.tab" {fa[$1]=$2; next} {fa[$1]=substr(fa[$1], 1, $2-1) $4 substr(fa[$1], $2+1, length(fa[$1])-$2)} END {for (id in fa){print ">" id "\n" fa[id]}}' /data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.tab /lu213/brandon.lind/data/BURT/data/04_vcfs/TotalRawSNPs_alt-filtered_G_added-lengths.txt > /lu213/brandon.lind/data/BURT/data/04_vcfs/psuedogenome_G.fa 

awk 'FILENAME=="/data/database/Pita_db/Pita2_stitched/pita2_stitch_v2.tab" {fa[$1]=$2; next} {fa[$1]=substr(fa[$1], 1, $2-1) $4 substr(fa[$1],